In [3]:
import pandas as pd
import requests
import pandas_gbq
from google.oauth2 import service_account
from pandas.io.json import json_normalize

In [4]:
# set up the access to the GCP database, where we will store the listing data
keyfile='gbq_json/project-pulldata-a0e9dfe25409.json'
project_id = 'project-pulldata'

credentials = service_account.Credentials.from_service_account_file(
    keyfile,
)

pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = project_id

# COMPANY LISTING --> API EXTRACTION

In [5]:
##Company Listing
api = 'GetCompanyListing'
name = 'ns_media'
token_id = 'hPP7@MubwL1C750pkWgUieNorBovxMsito/HZ9TilA4='

page = 1
url = f'http://apidata.globaldata.com/CompanyIntegratedViewNSMG/api/Content/{api}?TokenID={token_id}&PageNumber={page}&api_key={token_id}'
response = requests.get(url).json()
# assign the number of pages to a variable
num_pages = response['NoOfPages']
# instantiate an empty dataframe
companies_df = pd.DataFrame()

# in a loop over the 7 pages of the API results, fill the dataframe with the respective resutls
for page in range(1, num_pages+1):
    # apply the complete API call url
    url = f'http://apidata.globaldata.com/CompanyIntegratedViewNSMG/api/Content/{api}?TokenID={token_id}&PageNumber={page}&api_key={token_id}'
    # GET the response json
    response = requests.get(url).json()
    # parse the JSON to a pandas dataframe
    curr_df = pd.DataFrame.from_dict(response['Records'])
    # fill the dataframe above with each response
    companies_df = pd.concat([companies_df, curr_df],sort=False)
companies_df['PublishedDate'] = pd.to_datetime(companies_df['PublishedDate'])   
companies_df.to_csv('BankingCompanyProfiles__CompanyListing_202008271110.csv')

# COMPANY DETAILS --> API EXTRACTION

In [67]:
## Company Details

api = 'GetCompanyDetails'
name = 'ns_media'
token_id = 'hPP7@MubwL1C750pkWgUieNorBovxMsito/HZ9TilA4'
page = 1
url = f'http://apidata.globaldata.com/CompanyIntegratedViewNSMG/api/Content/{api}?TokenID={token_id}%3D&PageNumber={page}&api_key={token_id}%3D'
response = requests.get(url).json()
# assign the number of pages to a variable
num_pages = response['NoOfPages']
# create an empty dataframe, which would contain all data for THIS city
details_df = pd.DataFrame()

# in a loop over the 3 pages of the API results, fill the dataframe with the respective resutls
for page in range(1,num_pages+1):
    # apply the complete API call url
    url = f'http://apidata.globaldata.com/CompanyIntegratedViewNSMG/api/Content/{api}?TokenID={token_id}%3D&PageNumber={page}&api_key={token_id}%3D'
    # GET the response json
    response = requests.get(url).json()
    # parse the JSON to a pandas dataframe
    curr_df = pd.DataFrame.from_dict(response['Records'])
    # fill the dataframe above with each response
    details_df = pd.concat([details_df, curr_df],sort=False)
details_df['PublishedDate'] = pd.to_datetime(details_df['PublishedDate'])
original_table = details_df.iloc[:,:-10]
original_table['SiteNames'] = details_df['SiteNames']
original_table[['MajorProducts','MajorServices','MajorBrands']] = details_df[['MajorProducts','MajorServices','MajorBrands']]
original_table.to_csv('BankingCompanyProfiles__CompanyDetails_202008271110.csv')

In [9]:
# loop for each company from details_df collect historical events
total_history = pd.DataFrame()
for company in details_df['CompanyId'].tolist():
    individual = details_df[details_df['CompanyId'] == company]
    history_table = json_normalize(individual['History'].tolist()[0])
    history_table['CompanyId'] = individual.iloc[0,0]
    total_history = pd.concat([total_history, history_table],sort=False)

total_history.to_csv('BankingCompanyProfiles__HistoryDetails_202008251740.csv')

In [114]:
# loop for each company from details_df collect LocationSubsidiaries
total_locationSubsidiaries = pd.DataFrame()
for company in details_df['CompanyId'].tolist():
    individual = details_df[details_df['CompanyId'] == company]
    if(individual['LocationSubsidiaries'].tolist()[0] != None):
        subsidiaries_table = json_normalize(individual['LocationSubsidiaries'].tolist()[0])
        subsidiaries_table['SubsidiariesId'] = individual.iloc[0,0]
        total_competitors = pd.concat([total_locationSubsidiaries, subsidiaries_table],sort=False)

total_locationSubsidiaries.to_csv('BankingCompanyProfiles__LocationSubsidiariesDetails_202008251745.csv')

In [52]:
# loop for each company from details_df collect KeyTopcompetitors
total_competitors = pd.DataFrame()
for company in details_df['CompanyId'].tolist():
    individual = details_df[details_df['CompanyId'] == company]
    competitors_table = json_normalize(individual['KeyTopcompetitors'].tolist()[0])
    competitors_table['CompetitorId'] = individual.iloc[0,0]
    total_competitors = pd.concat([total_competitors, competitors_table],sort=False)

total_competitors.to_csv('BankingCompanyProfiles__KeyTopCompetitorsDetails_202008251745.csv')

In [58]:
# loop for each company from details_df collect KeyEmployees
total_employees = pd.DataFrame()
for company in details_df['CompanyId'].tolist():
    individual = details_df[details_df['CompanyId'] == company]
    employee_table = json_normalize(individual['KeyEmployees'].tolist()[0])
    employee_table['CompanyId'] = individual.iloc[0,0]
    total_employees = pd.concat([total_employees, employee_table],sort=False)

total_employees.to_csv('BankingCompanyProfiles__KeyEmployeesDetails_202008251750.csv')

In [60]:
# loop for each company from details_df collect KeyFacts
total_facts = pd.DataFrame()
for company in details_df['CompanyId'].tolist():
    individual = details_df[details_df['CompanyId'] == company]
    facts_table = json_normalize(individual['KeyFacts'].tolist()[0])
    facts_table['CompanyId'] = individual.iloc[0,0]
    total_facts = pd.concat([total_facts, facts_table],sort=False)

total_facts.to_csv('BankingCompanyProfiles__KeyFactsDetails_202008251755.csv')

In [62]:
## missing: SwotAnalysis(companyid and Overview) which needs to be split in: SwotAnalysis_Strengths,
## SwotAnalysis_Weakness,SwotAnalysis_Opprotunities,SwotAnalysis_Threats

In [11]:
# loop for each company from details_df collect SwotAnalysis
total_swot = pd.DataFrame()
for company in details_df['CompanyId'].tolist():
    individual = details_df[details_df['CompanyId'] == company]
    swot_table = json_normalize(individual['SwotAnalysis'].tolist()[0])
    swot_table['CompanyId'] = individual.iloc[0,0]
    total_swot = pd.concat([total_swot, swot_table],sort=False)
    total_swot1 = total_swot[['Overview','CompanyId']]
total_swot1.to_csv('BankingCompanyProfiles__SwotAnalysisDetails_202008260900.csv')

In [12]:
# loop for each company from SwotAnalysis collect Strengths
total_strengths = pd.DataFrame()
for company in total_swot['CompanyId'].tolist():
    individual = total_swot[total_swot['CompanyId'] == company]
    strengths = json_normalize(individual['Strengths'].tolist()[0])
    strengths['CompanyId'] = company
    total_strengths = pd.concat([total_strengths, strengths],sort=False)
total_strengths.to_csv('BankingCompanyProfiles__StrengthsSwotAnalysisDetails_202008261615.csv')

In [13]:
# loop for each company from SwotAnalysis collect Weakness

total_weakness = pd.DataFrame()
for company in total_swot['CompanyId'].tolist():
    individual = total_swot[total_swot['CompanyId'] == company]
    weakness = json_normalize(individual['Weakness'].tolist()[0])
    weakness['CompanyId'] = company
    total_weakness = pd.concat([total_weakness, weakness],sort=False)
total_weakness.to_csv('BankingCompanyProfiles__WeaknessSwotAnalysisDetails_202008260.csv')

In [14]:
# loop for each company from SwotAnalysis collect Opprotunities
total_opprotunities = pd.DataFrame()
for company in total_swot['CompanyId'].tolist():
    individual = total_swot[total_swot['CompanyId'] == company]
    opportunities = json_normalize(individual['Opprotunities'].tolist()[0])
    opportunities['CompanyId'] = individual.iloc[0,0]
    total_opprotunities = pd.concat([total_opprotunities, opportunities],sort=False)
total_opprotunities.to_csv('BankingCompanyProfiles__OpportunitiesSwotAnalysisDetails_202008260915.csv')

In [15]:
# loop for each company from SwotAnalysis collect Threats
total_threats = pd.DataFrame()
for company in total_swot['CompanyId'].tolist():
    individual = total_swot[total_swot['CompanyId'] == company]
    threats = json_normalize(individual['Threats'].tolist()[0])
    threats['CompanyId'] = individual.iloc[0,0]
    total_threats = pd.concat([total_threats, threats],sort=False)
total_threats.to_csv('BankingCompanyProfiles__ThreatsSwotAnalysisDetails_202008260915.csv')

# DEAL DETAILS --> API EXTRACTION

In [14]:
## Deal Details

api = 'GetDealDetails'
name = 'ns_media'
token_id = 'hPP7@MubwL1C750pkWgUieNorBovxMsito/HZ9TilA4'

deal_details = pd.DataFrame()
#for each CompanyId
for company in companies_df['CompanyId'].tolist():
    page = 1
    ## call the api and collect number of pages
    url = f'http://apidata.globaldata.com/CompanyIntegratedViewNSMG/api/Content/{api}?TokenID={token_id}%3D&CompanyID={company}&api_key={token_id}%3D'
    # GET the response json
    response = requests.get(url).json()
    num_pages = response['NoOfPages']
    #print('Company: ',company,' pages: ' , num_pages, '\n')
    # in a loop over the num_pages pages of the API results, fill the dataframe with the respective resutls
    for page in range(1,num_pages+1):
        # apply the complete API call url
        url = f'http://apidata.globaldata.com/CompanyIntegratedViewNSMG/api/Content/{api}?TokenID={token_id}%3D&CompanyID={company}&PageNumber={page}&api_key={token_id}%3D'
        # GET the response json
        response = requests.get(url).json()
        # parse the JSON to a pandas dataframe
        curr_df = pd.DataFrame.from_dict(response['Records'])
        curr_df['CompanyId'] = company
        # fill the dataframe above with each response
        deal_details = pd.concat([deal_details, curr_df], sort=False)
deal_details['DealID'] = deal_details['DealID'].astype('int32')
## Fixing date 0001-01-01
for date in deal_details['DealCompletedDate'].tolist():
    if (date == '0001-01-01T00:00:00') :
        (deal_details['DealCompletedDate'][deal_details['DealCompletedDate'] == date]) = None        
for date in deal_details['AnnounceDate'].tolist():
    if (date == '0001-01-01T00:00:00') :
        (deal_details['AnnounceDate'][deal_details['AnnounceDate'] == date]) = None
for date in deal_details['OptionalDateValue'].tolist():
    if (date == '0001-01-01T00:00:00') :
        (deal_details['OptionalDateValue'][deal_details['OptionalDateValue'] == date]) = None
        
deal_details['DealCompletedDate'] = pd.to_datetime(deal_details['DealCompletedDate'])
deal_details['PublishedDate'] = pd.to_datetime(deal_details['PublishedDate'])
deal_details['AnnounceDate'] = pd.to_datetime(deal_details['AnnounceDate'])
deal_details['RumorDate'] = pd.to_datetime(deal_details['RumorDate'])
deal_details['OptionalDateValue'] = pd.to_datetime(deal_details['OptionalDateValue'])


original_deals = deal_details[['DealID','Title','Description','DealCompletedDate','DealCompletedYear','DealCompletedQuarter',
                               'DealStatus','DealType','DealValue','DealRationale','RoundofFinancing','DealCountry',
                               'UrlNode','PublishedDate','AnnounceDate','DealSubType','DealSubTypeL1','DealSubTypeL2',
                               'DealSubTypeL3','AcquiredStake','LocalCurrency','AssetName','RumorDate','OptionalDateName',
                               'OptionalDateValue','Sources','CompanyId']]
original_deals.to_csv('BankingCompanyProfiles__DealDetails_202008271240.csv')

In [42]:
# loop for each deal from deals collect TargetAssets
total_targetassets = pd.DataFrame()
for deal in deal_details['DealID'].tolist():
    individual = deal_details[deal_details['DealID'] == deal]
    if(individual['TargetAssets'].tolist()[0] != None):
        targetAssets = json_normalize(individual['TargetAssets'].tolist()[0])
        targetAssets['DealID'] = individual.iloc[0,0]
        total_targetassets = pd.concat([total_targetassets, targetAssets],sort=False)
total_targetassets.to_csv('BankingCompanyProfiles__TargetAssetsDealDetails_202008261120.csv')

In [122]:
# loop for each deal from deals collect Targets
total_targets = pd.DataFrame()
for deal in deal_details['DealID'].tolist():
    individual = deal_details[deal_details['DealID'] == deal]
    if(individual['Targets'].tolist()[0] != None):
        targets = json_normalize(individual['Targets'].tolist()[0])
        targets['DealID'] = individual.iloc[0,0]
        total_targets = pd.concat([total_targets, targets],sort=False)
total_targets['CompanyId'] = total_targets['CompanyId'].astype('int32')
total_targets.to_csv('BankingCompanyProfiles__TargetsDealDetails_202008261120.csv')

IndexError: list index out of range

In [48]:
# loop for each deal from deals collect Aquirers
total_aquirers = pd.DataFrame()
for deal in deal_details['DealID'].tolist():
    individual = deal_details[deal_details['DealID'] == deal]
    if(individual['Aquirers'].tolist()[0] != None):
        aquirers = json_normalize(individual['Aquirers'].tolist()[0])
        aquirers['DealID'] = individual.iloc[0,0]
        total_aquirers = pd.concat([total_aquirers, aquirers], sort=False)
total_aquirers['CompanyId'] = total_aquirers['CompanyId'].astype('int32')
total_aquirers.to_csv('BankingCompanyProfiles__AquirersDealDetails_202008261125.csv')

In [82]:
# loop for each deal from deals collect Vendors
total_vendors = pd.DataFrame()
for deal in deal_details['DealID'].tolist():
    individual = deal_details[deal_details['DealID'] == deal]
    if(individual['Vendors'].tolist()[0] != None):
        vendors = json_normalize(individual['Vendors'].tolist()[0])
        vendors['DealID'] = individual.iloc[0,0]
        total_vendors = pd.concat([total_vendors, vendors], sort=False)
total_vendors['CompanyId'] = total_vendors['CompanyId'].astype('int32')
total_vendors.to_csv('BankingCompanyProfiles__VendorsDealDetails_202008261125.csv')

In [92]:
import numpy as np
# MajorProducts,MajorServices,MajorBrands,LocationSubsidiaries
columnlist1 = set(deal_details['Vendors'].tolist())
print(columnlist1)

{None}


In [124]:
# loop for each deal from deals collect Entities
total_entities = pd.DataFrame()
for deal in deal_details['DealID'].tolist():
    individual = deal_details[deal_details['DealID'] == deal]
    if(individual['Entities'].tolist()[0] != None):
        entities = json_normalize(individual['Entities'].tolist()[0])
        entities['DealID'] = individual.iloc[0,0]
        total_entities = pd.concat([total_entities, entities], sort=False)
total_entities.to_csv('BankingCompanyProfiles__EntitiesDealDetails_202008261125.csv')

IndexError: list index out of range

In [52]:
# loop for each deal from deals collect FinacialAdvisors
total_finacialAdvisors = pd.DataFrame()
for deal in deal_details['DealID'].tolist():
    individual = deal_details[deal_details['DealID'] == deal]
    if(individual['FinacialAdvisors'].tolist()[0] != None):
        FinacialAdvisors = json_normalize(individual['FinacialAdvisors'].tolist()[0])
        FinacialAdvisors['DealID'] = individual.iloc[0,0]
        total_finacialAdvisors = pd.concat([total_finacialAdvisors, FinacialAdvisors], sort=False)
total_finacialAdvisors.to_csv('BankingCompanyProfiles__FinacialAdvisorsDealDetails_202008261130.csv')

In [54]:
# loop for each deal from deals collect LegalAdvisors
total_legalAdvisors = pd.DataFrame()
for deal in deal_details['DealID'].tolist():
    individual = deal_details[deal_details['DealID'] == deal]
    if(individual['LegalAdvisors'].tolist()[0] != None):
        LegalAdvisors = json_normalize(individual['LegalAdvisors'].tolist()[0])
        LegalAdvisors['DealID'] = individual.iloc[0,0]
        total_legalAdvisors = pd.concat([total_legalAdvisors, LegalAdvisors], sort=False)
total_legalAdvisors.to_csv('BankingCompanyProfiles__LegalAdvisorsDealDetails_202008261135.csv')

# NEWS DETAILS --> API EXTRACTION

In [ ]:
## News Details

api = 'GetNewsDetails'
name = 'ns_media'
token_id = 'hPP7@MubwL1C750pkWgUieNorBovxMsito/HZ9TilA4'

news_details = pd.DataFrame()
#for each CompanyId
for company in companies_df['CompanyId'].tolist():
    page = 1
    ## call the api and collect number of pages
    url = f'http://apidata.globaldata.com/CompanyIntegratedViewNSMG/api/Content/{api}?TokenID={token_id}%3D&CompanyID={company}&api_key={token_id}%3D'
    # GET the response json
    response = requests.get(url).json()
    num_pages = response['NoOfPages']
    #print('Company: ',company,' pages: ' , num_pages, '\n')
    # in a loop over the num_pages pages of the API results, fill the dataframe with the respective resutls
    for page in range(1,num_pages+1):
        # apply the complete API call url
        url = f'http://apidata.globaldata.com/CompanyIntegratedViewNSMG/api/Content/{api}?TokenID={token_id}%3D&CompanyID={company}&PageNumber={page}&api_key={token_id}%3D'     
        # GET the response json
        response = requests.get(url).json()
        # parse the JSON to a pandas dataframe
        curr_df = pd.DataFrame.from_dict(response['Records'])
        curr_df['CompanyId'] = company
        # fill the dataframe above with each response
        news_details = pd.concat([news_details, curr_df], sort=False)
news_details['PublishedDate'] = pd.to_datetime(news_details['PublishedDate'])
original_news = news_details.drop(columns=['NewsArticleCompanies'])
original_news.to_csv('BankingCompanyProfiles__NewsDetails_202008271210.csv')

In [103]:
# loop for each deal from deals collect NewsArticleCompanies
total_NewsArticleCompanies = pd.DataFrame()
for new in news_details['NewsArticleId'].tolist():
    individual = news_details[news_details['NewsArticleId'] == new]
    if(individual['NewsArticleCompanies'].tolist()[0] != None):
        newsArticleCompanies = json_normalize(individual['NewsArticleCompanies'].tolist()[0])
        newsArticleCompanies['NewsArticleId'] = individual.iloc[0,0]
        total_NewsArticleCompanies = pd.concat([total_NewsArticleCompanies, newsArticleCompanies], sort=False)
total_NewsArticleCompanies.to_csv('BankingCompanyProfiles__NewsArticleCompaniesDetails_202008261155.csv')